# Google Colab Init

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !pip install transformers==2.8.0 
# !pip install deeppavlov

# !pip uninstall -y tensorflow tensorflow-gpu
# !pip install tensorflow-gpu==1.15.2

# !python -m deeppavlov install squad_bert

# !pip uninstall -y scikit-learn
# !pip install scikit-learn

# !pip install pandas

# RuBert DeepPavlov

In [3]:
import os
import pickle
import warnings
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from deeppavlov.core.common.file import read_json
from deeppavlov import build_model, configs, train_model
from deeppavlov.models.torch_bert.torch_transformers_classifier import TorchTransformersClassifierModel
from deeppavlov.models.preprocessors.torch_transformers_preprocessor import TorchTransformersPreprocessor
from sklearn.metrics import (f1_score, precision_score, average_precision_score, roc_auc_score,
                             classification_report, plot_roc_curve, accuracy_score, make_scorer,
                             plot_precision_recall_curve, precision_recall_curve, recall_score)
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tqdm.auto import tqdm

[nltk_data] Downloading package punkt to /home/kate/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/kate/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/kate/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/kate/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


In [4]:
warnings.filterwarnings('ignore')

## Get data

In [5]:
data_path = '/content/drive/MyDrive/SarcasmDetection/data/Quotes'
dataname = 'quotes_ling_feats.pickle'
with open(os.path.join(data_path, dataname), 'rb') as f:
    df = shuffle(pickle.load(f), random_state=8)

In [6]:
train_df, test_df = train_test_split(df, test_size=0.3, random_state=8)
train_df, valid_df = train_test_split(train_df, test_size=0.1, random_state=8)

In [7]:
train_df = train_df.groupby(
    'target', group_keys=False
).apply(lambda x: x.sample(n=(train_df.target == 1).sum())).sample(frac=1).reset_index(drop=True)

In [8]:
train_df[['quote', 'target']].to_csv('train.csv', index=False)
valid_df[['quote', 'target']].to_csv('valid.csv', index=False)
test_df[['quote', 'target']].to_csv('test.csv', index=False)

In [9]:
def chunks(list_like, n):
    for i in range(0, len(list_like), n):
        yield list_like[i:i + n]

In [10]:
def show_test_classification_metrics(y_test, y_pred, y_pred_prob, X_test=None, classifier=None):
    print(f"F1: {f1_score(y_test, y_pred):.5}")
    print(f"PREC: {precision_score(y_test, y_pred):.5}")
    print(f"PR-AUC: {average_precision_score(y_test, y_pred_prob):.5}")
    print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_prob):.5}")
    print('-------------------------------------------------------')
    print(classification_report(y_test, y_pred, labels=[0, 1]))
    print('-------------------------------------------------------')
    if classifier:
        fig, ax = plt.subplots(1, 2, figsize=(15, 5))
        ax[0].set_title('Precision-Recall curve')
        plot_precision_recall_curve(classifier, X_test, y_test, ax=ax[0])
        ax[1].set_title('ROC-AUC curve')
        plot_roc_curve(classifier, X_test, y_test, ax=ax[1])
        plt.show()

In [11]:
!python -m deeppavlov download paraphraser_rubert

2021-04-13 17:56:59.694 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'paraphraser_rubert' as '/usr/local/lib/python3.7/dist-packages/deeppavlov/configs/classifiers/paraphraser_rubert.json'
2021-04-13 17:57:00.295 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/datasets/paraphraser.zip?config=paraphraser_rubert download because of matching hashes
2021-04-13 17:57:16.496 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/classifiers/paraphraser_rubert_v0.tar.gz?config=paraphraser_rubert download because of matching hashes
2021-04-13 17:57:19.271 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1.tar.gz?config=paraphraser_rubert download because of matching hashes
2021-04-13 17:57:19.564 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/d

In [11]:
bert_config = read_json(configs.classifiers.rusentiment_bert)

bert_config['dataset_reader']['x'] = 'quote'
bert_config['dataset_reader']['y'] = 'target'
bert_config['dataset_reader']['data_path'] = './'
bert_config['dataset_reader']['train'] = 'train.csv'
bert_config['dataset_reader']['valid'] = 'valid.csv'
bert_config['dataset_reader']['test'] = 'test.csv'

del bert_config['dataset_iterator']['split_seed']
del bert_config['dataset_iterator']['field_to_split']
del bert_config['dataset_iterator']['split_fields']
del bert_config['dataset_iterator']['split_proportions']

bert_config['metadata']['variables']['MODEL_PATH'] = '{MODELS_PATH}/classifiers/bert_classifier_v2/'

del bert_config['chainer']['pipe'][-2:]
del bert_config['chainer']['pipe'][1]
bert_config['chainer']['pipe'][1]['in'] = 'y'
bert_config['chainer']['pipe'][1]['depth'] = 2
bert_config['chainer']['pipe'][2]['n_classes'] = 2
bert_config['train']['metrics'] = [bert_config['train']['metrics'][-1]]
bert_config['chainer']['out'] = ['y_pred_probas']
bert_config['train']['epochs'] = 2
bert_config['train']['batch_size'] = 32
bert_config['train']['show_examples'] = True

vocab_file = '{DOWNLOADS_PATH}/bert_models/rubert_cased_L-12_H-768_A-12_v1/vocab.txt'
bert_config_file = "{DOWNLOADS_PATH}/bert_models/rubert_cased_L-12_H-768_A-12_v1/bert_config.json"
pretrained_bert = "{DOWNLOADS_PATH}/bert_models/rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt"

bert_config['chainer']['pipe'][0]['vocab_file'] = vocab_file
bert_config['chainer']['pipe'][1]['bert_config_file'] = bert_config_file
bert_config['chainer']['pipe'][1]['pretrained_bert'] = pretrained_bert

bert_config['chainer']['pipe'][2]['bert_config_file'] = bert_config_file
bert_config['chainer']['pipe'][2]['pretrained_bert'] = pretrained_bert

In [12]:
m = train_model(bert_config)

2021-04-13 18:28:59.93 INFO in 'deeppavlov.core.trainers.fit_trainer'['fit_trainer'] at line 68: NNTrainer got additional init parameters ['pytest_max_batches'] that will be ignored:













The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.





Instructions for updating:
Use standard file APIs to check for files with this prefix.


2021-04-13 18:29:22.457 INFO in 'deeppavlov.models.bert.bert_classifier'['bert_classifier'] at line 99: [initializing model with Bert from /root/.deeppavlov/downloads/bert_models/rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt]



INFO:tensorflow:Restoring parameters from /root/.deeppavlov/downloads/bert_models/rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt


2021-04-13 18:33:24.850 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 199: Initial best roc_auc of 0.4665


...

{"valid": {"eval_examples_count": 27993, "metrics": {"roc_auc": 0.7787}, "time_spent": "0:45:22", "examples": [{"x": "— Удивительно, что они в дверь постучали. А не вломились...", "y_predicted": [0.17536188662052155, 0.8246381282806396], "y_true": "0"}, {"x": "Я счастлива лишь мечтаньем, Призрак прошлого ловлю: Вспоминаю ночь, свиданье... Но уж вновь не полюблю! Нет, нет, не хочу! Ничего я не хочу!", "y_predicted": [0.9078459143638611, 0.09215406328439713], "y_true": "0"}, {"x": "Он верил, что корабли приходят в гавань.", "y_predicted": [0.5906669497489929, 0.4093330204486847], "y_true": "0"}, {"x": "– Оказывается, это легко – убивать Помнишь Чушака? Олег убил его. И еще Налима. Или покалечил. Я тоже одного Мы боимся это вспоминать, Мрак, но мы убивали. И ничто у нас в душе не оборвалось, и гром с небес не сразил. Мы ушли через Болото, оставив позади трупы", "y_predicted": [0.868127167224884, 0.13187280297279358], "y_true": "0"}, {"x": "— Сколько можно есть? — Нужно много энергии, чтоб

2021-04-13 19:15:13.468 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/classifiers/bert_classifier_v2/model]


INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/classifiers/bert_classifier_v2/model
{"train": {"eval_examples_count": 34680, "metrics": {"roc_auc": 0.9139}, "time_spent": "0:05:02", "examples": [{"x": "Люди хотят, чтобы журавли всегда возвращались.", "y_predicted": [0.9000442028045654, 0.09995580464601517], "y_true": "0"}, {"x": "У актера все может быть ненастоящим, кроме страсти.", "y_predicted": [0.6182155609130859, 0.38178446888923645], "y_true": "0"}, {"x": "Ничего не может быть гаже, чем когда приехавший первым гость мыкается, не зная, что ему предпринять, а его законная мегера шёпотом пилит его за то, что они приехали раньше всех. Такие балы надо выбрасывать на помойку, королева.", "y_predicted": [0.16216744482517242, 0.837832510471344], "y_true": "1"}, {"x": "Когда мне говорят будь собой, я теряюсь и не знаю, кем из собой мне быть.", "y_predicted": [0.8619773983955383, 0.13802261650562286], "y_true": "0"}, {"x": "Лучше поговорите с ним. Мы не можем наверняка 

2021-04-13 19:42:03.274 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/classifiers/bert_classifier_v2/model]


INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/classifiers/bert_classifier_v2/model


In [13]:
preds_proba = []
for batch in tqdm(chunks(test_df["quote"].values, 64), total=int(test_df.index.size / 64)):
    preds_proba.append(m(batch))

preds = np.concatenate(preds_proba)

show_test_classification_metrics(
    test_df.target.values, 
    (preds[:, 1] > 0.5).astype(int), 
    preds[:, 1]
)


F1: 0.25756
PREC: 0.15932
PR-AUC: 0.19143
ROC-AUC: 0.77888
-------------------------------------------------------
              precision    recall  f1-score   support

           0       0.97      0.74      0.84    111669
           1       0.16      0.67      0.26      8299

    accuracy                           0.73    119968
   macro avg       0.56      0.70      0.55    119968
weighted avg       0.91      0.73      0.80    119968

-------------------------------------------------------
